In [1]:
# from connection_parameters import UCM_PUBLISHER, AXL_USER, AXL_PASSWORD

# or set them directly
UCM_PUBLISHER = 'us-cm-pub.ent-pa.com'
AXL_USER = 'administrator'
AXL_PASSWORD = 'W0rkAllN1ght'

# Using [Zeep](https://github.com/mvantellingen/python-zeep)

Zeep is a SOAP client for Python. Zeep inspects the WSDL document and generates the corresponding code to use the services and types in the document. This provides an easy to use programmatic interface to a SOAP server.




In [2]:
import requests
import os
import zeep
import zeep.cache
import zeep.plugins
import zeep.exceptions
import urllib3
from lxml import etree

# a simple zeep plugin to log all SOAP request and responses to stdin
class LoggingPlugin(zeep.plugins.Plugin):
    @staticmethod
    def print_envelope(header, envelope):
        s = etree.tostring(envelope, pretty_print=True).decode().strip()
        print('\n'.join(f'{header}:{l}' for l in s.splitlines()))

    def ingress(self, envelope, http_headers, operation):
        LoggingPlugin.print_envelope('in', envelope)
        return envelope, http_headers

    def egress(self, envelope, http_headers, operation, binding_options):
        LoggingPlugin.print_envelope('out', envelope)
        return envelope, http_headers
    
# disable warnings for HTTPS sessions w/ diabled cert validation
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

axl_url = f'https://{UCM_PUBLISHER}:8443/axl/'

wsdl_version = '11.5'
wsdl = os.path.join('WSDL', wsdl_version, 'AXLAPI.wsdl')
print(f'Using WSDL: {wsdl}')

session = requests.Session()
session.auth = (AXL_USER, AXL_PASSWORD)
session.verify = False

transport = zeep.Transport(session=session, cache=zeep.cache.SqliteCache())
history = zeep.plugins.HistoryPlugin()
client = zeep.Client(wsdl=wsdl,
                     transport=transport,
                     # create a chain of plugins to keep the history of request/response
                     # and log everything to stdout
                     plugins=[history, LoggingPlugin()])
service = client.create_service('{http://www.cisco.com/AXLAPIService/}AXLAPIBinding', axl_url)


Using WSDL: WSDL/11.5/AXLAPI.wsdl


In [3]:
tags = 'name clause description dialPlanWizardGenId partitionUsage'

r = service.listCss(searchCriteria={'name':'%'}, returnedTags = {t:'' for t in tags.split()}, first=5)

out:<soap-env:Envelope xmlns:soap-env="http://schemas.xmlsoap.org/soap/envelope/">
out:  <soap-env:Body>
out:    <ns0:listCss xmlns:ns0="http://www.cisco.com/AXL/API/11.5">
out:      <searchCriteria>
out:        <name>%</name>
out:      </searchCriteria>
out:      <returnedTags>
out:        <description></description>
out:        <clause></clause>
out:        <dialPlanWizardGenId></dialPlanWizardGenId>
out:        <partitionUsage></partitionUsage>
out:        <name></name>
out:      </returnedTags>
out:      <first>5</first>
out:    </ns0:listCss>
out:  </soap-env:Body>
out:</soap-env:Envelope>
in:<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/">
in:  <soapenv:Body>
in:    <ns:listCssResponse xmlns:ns="http://www.cisco.com/AXL/API/11.5">
in:      <return>
in:        <css uuid="{03E46D7C-E8EB-744F-BF22-2400E1919389}">
in:          <description>CSS for devices to call internal DNs</description>
in:          <clause>DN</clause>
in:          <dialPlanWizardGenId/

In [4]:
css_list = r['return'].css
print(css_list)

[{
    'description': 'CSS for devices to call internal DNs',
    'clause': 'DN',
    'dialPlanWizardGenId': None,
    'partitionUsage': 'General',
    'name': 'Device_CSS',
    'uuid': '{03E46D7C-E8EB-744F-BF22-2400E1919389}'
}, {
    'description': 'SJC CoS internal',
    'clause': 'DN:Directory URI:URI:ESN:onNetRemote:SJCIntra:USToE164:USEmergency',
    'dialPlanWizardGenId': None,
    'partitionUsage': 'General',
    'name': 'SJCInternal',
    'uuid': '{A927D63D-CBCD-C0E7-F48D-1D681B1BC15E}'
}, {
    'description': 'SJC CoS National',
    'clause': 'DN:Directory URI:URI:ESN:onNetRemote:SJCIntra:USToE164:USPSTNNational:USEmergency',
    'dialPlanWizardGenId': None,
    'partitionUsage': 'General',
    'name': 'SJCNational',
    'uuid': '{E5E6C1F5-213E-FD7E-F1F4-460DA08D178A}'
}, {
    'description': 'Inbound CSS for PSTN Gateways',
    'clause': 'DN',
    'dialPlanWizardGenId': None,
    'partitionUsage': 'General',
    'name': 'DN',
    'uuid': '{2F787DE2-2745-7DC3-2547-41516BE1172

In [5]:
print('\n\n'.join(f'{css.name}({css.uuid}): {css.clause}' for css in css_list))

Device_CSS({03E46D7C-E8EB-744F-BF22-2400E1919389}): DN

SJCInternal({A927D63D-CBCD-C0E7-F48D-1D681B1BC15E}): DN:Directory URI:URI:ESN:onNetRemote:SJCIntra:USToE164:USEmergency

SJCNational({E5E6C1F5-213E-FD7E-F1F4-460DA08D178A}): DN:Directory URI:URI:ESN:onNetRemote:SJCIntra:USToE164:USPSTNNational:USEmergency

DN({2F787DE2-2745-7DC3-2547-41516BE1172E}): DN

RTPInternational({D6A9C0C8-9797-7279-FAA4-82A8D5B516F8}): DN:Directory URI:URI:ESN:onNetRemote:RTPIntra:USToE164:USPSTNNational:PSTNInternational:B2B_URI:USEmergency


In [6]:
print('SOAP Request sent to UCM:')
print(etree.tostring(history.last_sent['envelope'], pretty_print=True).decode())
print('\nSOAP Response received from UCM:')
print(etree.tostring(history.last_received['envelope'], pretty_print=True).decode())

SOAP Request sent to UCM:
<soap-env:Envelope xmlns:soap-env="http://schemas.xmlsoap.org/soap/envelope/">
  <soap-env:Body>
    <ns0:listCss xmlns:ns0="http://www.cisco.com/AXL/API/11.5">
      <searchCriteria>
        <name>%</name>
      </searchCriteria>
      <returnedTags>
        <description></description>
        <clause></clause>
        <dialPlanWizardGenId></dialPlanWizardGenId>
        <partitionUsage></partitionUsage>
        <name></name>
      </returnedTags>
      <first>5</first>
    </ns0:listCss>
  </soap-env:Body>
</soap-env:Envelope>


SOAP Response received from UCM:
<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/">
  <soapenv:Body>
    <ns:listCssResponse xmlns:ns="http://www.cisco.com/AXL/API/11.5">
      <return>
        <css uuid="{03E46D7C-E8EB-744F-BF22-2400E1919389}">
          <description>CSS for devices to call internal DNs</description>
          <clause>DN</clause>
          <dialPlanWizardGenId/>
          <partitionUsage>Gen

# Executing SQL Queries

The [database dictionary](https://developer.cisco.com/docs/axl/#!12-5-cucm-data-dictionary) describes all tables in the UCM database and the relations between these tables.

The [executeSQLQuery](https://pubhub.devnetcloud.com/media/axl-schema-reference/docs/Files/AXLSoap_executeSQLQuery.html#Link45E) and [updateSQLQuery](https://pubhub.devnetcloud.com/media/axl-schema-reference/docs/Files/AXLSoap_executeSQLUpdate.html#Link466) methods enable direct access to the UCM database.

In [7]:
r = service.executeSQLQuery(sql='select * from processnode')

out:<soap-env:Envelope xmlns:soap-env="http://schemas.xmlsoap.org/soap/envelope/">
out:  <soap-env:Body>
out:    <ns0:executeSQLQuery xmlns:ns0="http://www.cisco.com/AXL/API/11.5">
out:      <sql>select * from processnode</sql>
out:    </ns0:executeSQLQuery>
out:  </soap-env:Body>
out:</soap-env:Envelope>
in:<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/">
in:  <soapenv:Body>
in:    <ns:executeSQLQueryResponse xmlns:ns="http://www.cisco.com/AXL/API/11.5">
in:      <return>
in:        <row>
in:          <pkid>00000000-1111-0000-0000-000000000000</pkid>
in:          <name>EnterpriseWideData</name>
in:          <mac/>
in:          <systemnode>t</systemnode>
in:          <description/>
in:          <isactive>t</isactive>
in:          <nodeid>1</nodeid>
in:          <tknodeusage>1</tknodeusage>
in:          <ipv6name/>
in:          <fklbmhubgroup/>
in:          <tkprocessnoderole>1</tkprocessnoderole>
in:          <tkssomode>0</tkssomode>
in:        </row>
in:   

In [8]:
rows = r['return'].row
rows

[[<Element pkid at 0x7f930b300d48>,
  <Element name at 0x7f930b300d88>,
  <Element mac at 0x7f930b300dc8>,
  <Element systemnode at 0x7f930b300e08>,
  <Element description at 0x7f930b300e48>,
  <Element isactive at 0x7f930b300e88>,
  <Element nodeid at 0x7f930b300ec8>,
  <Element tknodeusage at 0x7f930b300f08>,
  <Element ipv6name at 0x7f930b300f48>,
  <Element fklbmhubgroup at 0x7f930b300f88>,
  <Element tkprocessnoderole at 0x7f930b300fc8>,
  <Element tkssomode at 0x7f930b309048>],
 [<Element pkid at 0x7f930b309088>,
  <Element name at 0x7f930b3090c8>,
  <Element mac at 0x7f930b309108>,
  <Element systemnode at 0x7f930b309148>,
  <Element description at 0x7f930b309188>,
  <Element isactive at 0x7f930b3091c8>,
  <Element nodeid at 0x7f930b309208>,
  <Element tknodeusage at 0x7f930b309248>,
  <Element ipv6name at 0x7f930b309288>,
  <Element fklbmhubgroup at 0x7f930b3092c8>,
  <Element tkprocessnoderole at 0x7f930b309308>,
  <Element tkssomode at 0x7f930b309348>],
 [<Element pkid at 0x7

Not very helpful. We'd prefer to have a list of objects which allow to access the returned values as attributes

In [3]:
class Row:
    # Helper class to create an object based on a row returned from an AXL executeSqlQuery
    def __init__(self, row):
        # save a dictionary of key/value pairs
        self._obj =  {e.tag:e.text for e in row}

    def __repr__(self):
        """
        String representation for row object
        :return:
        """
        return f'Row({", ".join(f"{k}={v}" for k,v in self._obj.items())})'

    def __getattr__(self, item):
        """
        Satisfy attribute access by accessing saved directory
        :param item:
        :return:
        """
        return self._obj[item]

In [10]:
rows = [Row(r) for r in rows]

The `Row` helper class also allows us to access the values in the rows as attributes.

In [11]:
names = [r.name for r in rows]
names

['EnterpriseWideData',
 'ucm-pub.dcloud.cisco.com',
 'ucm-sub1.dcloud.cisco.com',
 'imp-pub.dcloud.cisco.com']

... and the `__repr__` method implementation gives us a readable representation of the rows

In [12]:
print('\n'.join(str(r) for r in rows))

Row(pkid=00000000-1111-0000-0000-000000000000, name=EnterpriseWideData, mac=None, systemnode=t, description=None, isactive=t, nodeid=1, tknodeusage=1, ipv6name=None, fklbmhubgroup=None, tkprocessnoderole=1, tkssomode=0)
Row(pkid=f827a48a-eb7c-424d-a94d-454e9f48318f, name=ucm-pub.dcloud.cisco.com, mac=None, systemnode=f, description=None, isactive=t, nodeid=2, tknodeusage=0, ipv6name=None, fklbmhubgroup=None, tkprocessnoderole=1, tkssomode=0)
Row(pkid=cb4b06b9-223c-4ffb-b63b-cd7ec38126d0, name=ucm-sub1.dcloud.cisco.com, mac=None, systemnode=f, description=None, isactive=t, nodeid=3, tknodeusage=1, ipv6name=None, fklbmhubgroup=None, tkprocessnoderole=1, tkssomode=0)
Row(pkid=7545a76a-35f8-3581-6d61-4b36618be0a4, name=imp-pub.dcloud.cisco.com, mac=None, systemnode=f, description=None, isactive=t, nodeid=5, tknodeusage=0, ipv6name=None, fklbmhubgroup=None, tkprocessnoderole=2, tkssomode=0)


Here's how we could get a list of CSSes using thin AXL. This is (more or less) equivalent to the listCss above.

In [13]:
r = service.executeSQLQuery(sql='select name,clause from callingsearchspace')
rows = [Row(r) for r in r['return'].row]

out:<soap-env:Envelope xmlns:soap-env="http://schemas.xmlsoap.org/soap/envelope/">
out:  <soap-env:Body>
out:    <ns0:executeSQLQuery xmlns:ns0="http://www.cisco.com/AXL/API/11.5">
out:      <sql>select name,clause from callingsearchspace</sql>
out:    </ns0:executeSQLQuery>
out:  </soap-env:Body>
out:</soap-env:Envelope>
in:<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/">
in:  <soapenv:Body>
in:    <ns:executeSQLQueryResponse xmlns:ns="http://www.cisco.com/AXL/API/11.5">
in:      <return>
in:        <row>
in:          <name>Device_CSS</name>
in:          <clause>DN</clause>
in:        </row>
in:        <row>
in:          <name>SJCInternal</name>
in:          <clause>DN:Directory URI:URI:ESN:onNetRemote:SJCIntra:USToE164:USEmergency</clause>
in:        </row>
in:        <row>
in:          <name>SJCNational</name>
in:          <clause>DN:Directory URI:URI:ESN:onNetRemote:SJCIntra:USToE164:USPSTNNational:USEmergency</clause>
in:        </row>
in:        <row>


In [14]:
print(rows)

[Row(name=Device_CSS, clause=DN), Row(name=SJCInternal, clause=DN:Directory URI:URI:ESN:onNetRemote:SJCIntra:USToE164:USEmergency), Row(name=SJCNational, clause=DN:Directory URI:URI:ESN:onNetRemote:SJCIntra:USToE164:USPSTNNational:USEmergency), Row(name=DN, clause=DN), Row(name=RTPInternational, clause=DN:Directory URI:URI:ESN:onNetRemote:RTPIntra:USToE164:USPSTNNational:PSTNInternational:B2B_URI:USEmergency), Row(name=RTPNational, clause=DN:Directory URI:URI:ESN:onNetRemote:RTPIntra:USToE164:USPSTNNational:USEmergency), Row(name=RTPInternal, clause=DN:Directory URI:URI:ESN:onNetRemote:RTPIntra:USToE164:USEmergency), Row(name=RCDInternal, clause=DN:Directory URI:URI:ESN:onNetRemote:RCDIntra:USToE164:USEmergency), Row(name=RCDNational, clause=DN:Directory URI:URI:ESN:onNetRemote:RCDIntra:USToE164:USPSTNNational:USEmergency), Row(name=UnityConnection, clause=DN:Directory URI:URI:ESN:onNetRemote:USToE164), Row(name=DefaultSubscribe, clause=DN:ESN:URI:onNetRemote:Directory URI), Row(name=T

Using simple thin AXL request we can easily execute some validations on the configured CSSes

For example: **Are there any CSSes without partitions assigned to them?**

In [15]:
csses_wo_partitions = [css.name for css in rows if css.clause is None]
print(f'CSSes w/o partitions: {", ".join(csses_wo_partitions)}')

CSSes w/o partitions: cms1-css, cms2-css


.. or: **Are there partitions which are not used in any CSS?**

In [16]:
import itertools

# let's try to determine the set of partitions used in CSSes
clauses = (r.clause for r in rows)

# only not empty clauses
clauses = (c for c in clauses if c is not None)

# get partition names by splitting at ':'
clauses = (c.split(':') for c in clauses)

# now chain all partition names, put them in a set to make them unique, and put them into a dictionary
# so that it's fast to check for existence of a partition
used_partitions = {p:'' for p in set(itertools.chain.from_iterable(clauses))}

# Get all partition names
r = service.executeSQLQuery(sql='select name from routepartition')
partition_names = [row[0].text for row in r['return'].row]

out:<soap-env:Envelope xmlns:soap-env="http://schemas.xmlsoap.org/soap/envelope/">
out:  <soap-env:Body>
out:    <ns0:executeSQLQuery xmlns:ns0="http://www.cisco.com/AXL/API/11.5">
out:      <sql>select name from routepartition</sql>
out:    </ns0:executeSQLQuery>
out:  </soap-env:Body>
out:</soap-env:Envelope>
in:<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/">
in:  <soapenv:Body>
in:    <ns:executeSQLQueryResponse xmlns:ns="http://www.cisco.com/AXL/API/11.5">
in:      <return>
in:        <row>
in:          <name>Directory URI</name>
in:        </row>
in:        <row>
in:          <name>Global Learned Enterprise Numbers</name>
in:        </row>
in:        <row>
in:          <name>Global Learned E164 Numbers</name>
in:        </row>
in:        <row>
in:          <name>Global Learned Enterprise Patterns</name>
in:        </row>
in:        <row>
in:          <name>Global Learned E164 Patterns</name>
in:        </row>
in:        <row>
in:          <name>PSTNInt

In [17]:
# are there any partitions which are not used in any CSS?
unused_partitions = [p for p in partition_names if p not in used_partitions]
print(f'Partitions not used in any CSS: {", ".join(unused_partitions)}')

Partitions not used in any CSS: Global Learned Enterprise Numbers, Global Learned E164 Numbers, Global Learned Enterprise Patterns, Global Learned E164 Patterns, USGWLocalizeCd, RTPGWLocalizeCn, SJCGWLocalizeCn, RCDGWLocalizeCn, blockmobile


In principle complex SQL statements can be used to execute more advanced queries. Multiple tables can be joined using the information published in the data dictionary:

## Table Relationships

**pkid** is the primary key ID. It is always of type GUID (a 36 character UUID).

Fields that begin with the letters "**fk**" represent foriegn keys into another table. The name of the field following the "fk" prefix up to but not including an underscore character is the name of the related table. The field in related table is always pkid. and is a GUID.

Fields that begin with the letters "**ik**" represent internal keys into the same table.

Fields that begin with a "**tk**" represent an enumerated type. This field is related to a table whose name begins with "Type" and ends with the name of the field following the prefix up to but not including an underscore character. The field in the related table is always "enum" and is an integer.


Using this information this SQL query combines the `numplan` and `typepatternusage` tables and selects a list of directory numbers starting with +1919 in ascending order:
```
select dnorpattern from numplan np left join typepatternusage tpu on np.tkpatternusage=tpu.enum where tpu.name="Device" and dnorpattern like "%+1919%" order by dnorpattern
```

In [18]:
r = service.executeSQLQuery('select dnorpattern,tkpatternusage from numplan np left join typepatternusage tpu on np.tkpatternusage=tpu.enum where tpu.name="Device" and dnorpattern like "%+1919%" order by dnorpattern')

out:<soap-env:Envelope xmlns:soap-env="http://schemas.xmlsoap.org/soap/envelope/">
out:  <soap-env:Body>
out:    <ns0:executeSQLQuery xmlns:ns0="http://www.cisco.com/AXL/API/11.5">
out:      <sql>select dnorpattern,tkpatternusage from numplan np left join typepatternusage tpu on np.tkpatternusage=tpu.enum where tpu.name="Device" and dnorpattern like "%+1919%" order by dnorpattern</sql>
out:    </ns0:executeSQLQuery>
out:  </soap-env:Body>
out:</soap-env:Envelope>
in:<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/">
in:  <soapenv:Body>
in:    <ns:executeSQLQueryResponse xmlns:ns="http://www.cisco.com/AXL/API/11.5">
in:      <return>
in:        <row>
in:          <dnorpattern>\+19195551020</dnorpattern>
in:          <tkpatternusage>2</tkpatternusage>
in:        </row>
in:        <row>
in:          <dnorpattern>\+19195551023</dnorpattern>
in:          <tkpatternusage>2</tkpatternusage>
in:        </row>
in:        <row>
in:          <dnorpattern>\+19195551026</d

In [19]:
dns = [r[0].text for r in r['return'].row]
print(dns)

['\\+19195551020', '\\+19195551023', '\\+19195551026', '\\+19195551055']


Knowing the enum for DNs the above query obviously could also have been simplified to only using the numplan table .. and no join.

In [14]:
r = service.executeSQLQuery(('select dnorpattern from numplan '
                             'where tkpatternusage=2 and '
                             'dnorpattern like "%+1919%" '
                             'order by dnorpattern'))

out:<soap-env:Envelope xmlns:soap-env="http://schemas.xmlsoap.org/soap/envelope/">
out:  <soap-env:Body>
out:    <ns0:executeSQLQuery xmlns:ns0="http://www.cisco.com/AXL/API/11.5">
out:      <sql>select dnorpattern from numplan where tkpatternusage=2 and dnorpattern like "%+1919%" order by dnorpattern</sql>
out:    </ns0:executeSQLQuery>
out:  </soap-env:Body>
out:</soap-env:Envelope>
in:<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/">
in:  <soapenv:Body>
in:    <ns:executeSQLQueryResponse xmlns:ns="http://www.cisco.com/AXL/API/11.5">
in:      <return>
in:        <row>
in:          <dnorpattern>\+19195551000</dnorpattern>
in:        </row>
in:        <row>
in:          <dnorpattern>\+19195551001</dnorpattern>
in:        </row>
in:        <row>
in:          <dnorpattern>\+19195551002</dnorpattern>
in:        </row>
in:        <row>
in:          <dnorpattern>\+19195551003</dnorpattern>
in:        </row>
in:        <row>
in:          <dnorpattern>\+19195551004<

In [21]:
dns = [r[0].text for r in r['return'].row]
print(dns)

['\\+19195551020', '\\+19195551023', '\\+19195551026', '\\+19195551055']


### Getting device counts per device type

In [4]:
r = service.executeSQLQuery(('select count(d.name) c, tm.name from device d '
                             'inner join typemodel tm on d.tkmodel=tm.enum '
                             'where tm.tkclass=1 '
                             'group by tm.name '
                             'order by tm.name'))

out:<soap-env:Envelope xmlns:soap-env="http://schemas.xmlsoap.org/soap/envelope/">
out:  <soap-env:Body>
out:    <ns0:executeSQLQuery xmlns:ns0="http://www.cisco.com/AXL/API/11.5">
out:      <sql>select count(d.name) c, tm.name from device d inner join typemodel tm on d.tkmodel=tm.enum where tm.tkclass=1 group by tm.name order by tm.name</sql>
out:    </ns0:executeSQLQuery>
out:  </soap-env:Body>
out:</soap-env:Envelope>
in:<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/">
in:  <soapenv:Body>
in:    <ns:executeSQLQueryResponse xmlns:ns="http://www.cisco.com/AXL/API/11.5">
in:      <return>
in:        <row>
in:          <c>11</c>
in:          <name>CTI Port</name>
in:        </row>
in:        <row>
in:          <c>34</c>
in:          <name>CTI Remote Device</name>
in:        </row>
in:        <row>
in:          <c>4</c>
in:          <name>Cisco 7821</name>
in:        </row>
in:        <row>
in:          <c>1</c>
in:          <name>Cisco 7861</name>
in:        

In [5]:
rows = [Row(r) for r in r['return'].row]
max_name = max(len(r.name) for r in rows)
print('\n'.join(f'{r.name:{max_name}} : {r.c}' for r in rows))

CTI Port                                : 11
CTI Remote Device                       : 34
Cisco 7821                              : 4
Cisco 7861                              : 1
Cisco 7960                              : 2
Cisco 7962                              : 1
Cisco 7965                              : 2
Cisco 7971                              : 1
Cisco 7975                              : 1
Cisco 8811                              : 3
Cisco 8821                              : 2
Cisco 8831                              : 3
Cisco 8832                              : 2
Cisco 8841                              : 3
Cisco 8845                              : 13
Cisco 8851                              : 3
Cisco 8861                              : 3
Cisco 8865                              : 17
Cisco 8945                              : 2
Cisco 8961                              : 1
Cisco 9951                              : 4
Cisco 9971                              : 22
Cisco ATA 190              

### Device counts per device pool and device type

In [9]:
r = service.executeSQLQuery(('select count(d.name) c, dp.name dpname, tm.name from device d '
                             'inner join typemodel tm on d.tkmodel=tm.enum '
                             'inner join devicepool dp on d.fkdevicepool=dp.pkid '
                             'where tm.tkclass=1 '
                             'group by dpname, tm.name '
                             'order by dpname, tm.name'))

out:<soap-env:Envelope xmlns:soap-env="http://schemas.xmlsoap.org/soap/envelope/">
out:  <soap-env:Body>
out:    <ns0:executeSQLQuery xmlns:ns0="http://www.cisco.com/AXL/API/11.5">
out:      <sql>select count(d.name) c, dp.name dpname, tm.name from device d inner join typemodel tm on d.tkmodel=tm.enum inner join devicepool dp on d.fkdevicepool=dp.pkid where tm.tkclass=1 group by dpname, tm.name order by dpname, tm.name</sql>
out:    </ns0:executeSQLQuery>
out:  </soap-env:Body>
out:</soap-env:Envelope>
in:<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/">
in:  <soapenv:Body>
in:    <ns:executeSQLQueryResponse xmlns:ns="http://www.cisco.com/AXL/API/11.5">
in:      <return>
in:        <row>
in:          <c>1</c>
in:          <dpname>Default</dpname>
in:          <name>CTI Remote Device</name>
in:        </row>
in:        <row>
in:          <c>1</c>
in:          <dpname>Default</dpname>
in:          <name>Cisco 8811</name>
in:        </row>
in:        <row>
in:  

In [12]:
rows = [Row(r) for r in r['return'].row]
max_name = max(len(r.name) for r in rows)
max_dp = max(len(r.dpname) for r in rows)
print('\n'.join(f'{r.dpname:{max_dp}}/{r.name:{max_name}} : {r.c}' for r in rows))

Default                                         /CTI Remote Device                       : 1
Default                                         /Cisco 8811                              : 1
Default                                         /Cisco 8845                              : 3
Default                                         /Cisco DX650                             : 1
Default                                         /Cisco TelePresence DX70                 : 1
Default                                         /Universal Device Template               : 2
RB1_Video_20MB                                  /Cisco DX650                             : 1
RB1_Video_encryptedSRST_20MB                    /Cisco 7975                              : 1
RB1_Video_encryptedSRST_20MB                    /Cisco 8865                              : 2
RB1_Video_encryptedSRST_20MB                    /Cisco 9971                              : 1
RB1_Video_encryptedSRST_20MB                    /Cisco DX650          

### More SQL Queries

More examples can be found here: https://www.cisco.com/c/en/us/support/docs/unified-communications/unified-communications-manager-callmanager/117726-technote-cucm-00.html

# Adding Users

In [13]:
# list some users
r = service.listUser(searchCriteria={'lastName':'%'}, returnedTags={'userid':''}, first=10)
user_list = r['return'].user
print(user_list) 

out:<soap-env:Envelope xmlns:soap-env="http://schemas.xmlsoap.org/soap/envelope/">
out:  <soap-env:Body>
out:    <ns0:listUser xmlns:ns0="http://www.cisco.com/AXL/API/11.5">
out:      <searchCriteria>
out:        <lastName>%</lastName>
out:      </searchCriteria>
out:      <returnedTags>
out:        <userid></userid>
out:      </returnedTags>
out:      <first>10</first>
out:    </ns0:listUser>
out:  </soap-env:Body>
out:</soap-env:Envelope>
in:<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/">
in:  <soapenv:Body>
in:    <ns:listUserResponse xmlns:ns="http://www.cisco.com/AXL/API/11.5">
in:      <return>
in:        <user uuid="{01159FCB-206F-3E99-331D-1E912AE45D8B}">
in:          <userid>nwheeler</userid>
in:        </user>
in:        <user uuid="{0137B722-B4BD-4926-E4E8-D4A8A12B413E}">
in:          <userid>nmcdonal</userid>
in:        </user>
in:        <user uuid="{013F34E9-D7AE-6FFD-D60A-755405FCD9DF}">
in:          <userid>bharrell</userid>
in:        </use

In [23]:
# get details of 1st user
user = user_list[0]
rt = 'firstName lastName userid presenceGroupName'
r = service.getUser(uuid=user.uuid, returnedTags={t:'' for t in rt.split()})
user_details = r['return'].user
print('\n'.join(f'{t}={user_details[t]}' for t in rt.split()))

out:<soap-env:Envelope xmlns:soap-env="http://schemas.xmlsoap.org/soap/envelope/">
out:  <soap-env:Body>
out:    <ns0:getUser xmlns:ns0="http://www.cisco.com/AXL/API/11.5">
out:      <uuid>{312750D9-0459-A367-52A7-F905A42F2648}</uuid>
out:      <returnedTags>
out:        <firstName></firstName>
out:        <lastName></lastName>
out:        <userid></userid>
out:        <presenceGroupName></presenceGroupName>
out:      </returnedTags>
out:    </ns0:getUser>
out:  </soap-env:Body>
out:</soap-env:Envelope>
in:<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/">
in:  <soapenv:Body>
in:    <ns:getUserResponse xmlns:ns="http://www.cisco.com/AXL/API/11.5">
in:      <return>
in:        <user uuid="{312750D9-0459-A367-52A7-F905A42F2648}">
in:          <firstName>Lucy</firstName>
in:          <displayName/>
in:          <lastName>Abbot</lastName>
in:          <userid>labbot</userid>
in:          <presenceGroupName uuid="{AD243D17-98B4-4118-8FEB-5FF2E1B781AC}">Standard Pre

In [24]:
# create a user object based on the type definition in the WSDL
factory = client.type_factory('ns0')
new_user = factory.XUser(
    firstName='Bob',
    lastName='Barnaby',
    userid='bbarnaby',
    presenceGroupName='Standard Presence group',
    # associated groups is a list of userGroup elements. Each userGroup has a name
    associatedGroups={
        'userGroup': [
            {'name': 'Standard CCM End Users'},
            {'name': 'Standard CTI Enabled'}
        ]
    }
)


Now try to add the user using the [addUser](https://pubhub.devnetcloud.com/media/axl-schema-reference/docs/Files/AXLSoap_AddUserReq.html#Link1EB) method

In [25]:
r = service.addUser(user=new_user)

out:<soap-env:Envelope xmlns:soap-env="http://schemas.xmlsoap.org/soap/envelope/">
out:  <soap-env:Body>
out:    <ns0:addUser xmlns:ns0="http://www.cisco.com/AXL/API/11.5">
out:      <user>
out:        <firstName>Bob</firstName>
out:        <lastName>Barnaby</lastName>
out:        <userid>bbarnaby</userid>
out:        <associatedGroups>
out:          <userGroup>
out:            <name>Standard CCM End Users</name>
out:          </userGroup>
out:          <userGroup>
out:            <name>Standard CTI Enabled</name>
out:          </userGroup>
out:        </associatedGroups>
out:        <presenceGroupName>Standard Presence group</presenceGroupName>
out:      </user>
out:    </ns0:addUser>
out:  </soap-env:Body>
out:</soap-env:Envelope>
in:<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/">
in:  <soapenv:Body>
in:    <ns:addUserResponse xmlns:ns="http://www.cisco.com/AXL/API/11.5">
in:      <return>{F28BE8F6-B8F0-A00C-6E58-926A1A991DCB}</return>
in:    </ns:addUser

In [26]:
new_user_uuid=r['return']
new_user_uuid

'{F28BE8F6-B8F0-A00C-6E58-926A1A991DCB}'

We successfully added a user using AXL. Verify the success by checking the endusers on UCM.

# AXL Errors

Let's try to add the same(!) user again and see what happens...


In [27]:
r = service.addUser(user=new_user)

out:<soap-env:Envelope xmlns:soap-env="http://schemas.xmlsoap.org/soap/envelope/">
out:  <soap-env:Body>
out:    <ns0:addUser xmlns:ns0="http://www.cisco.com/AXL/API/11.5">
out:      <user>
out:        <firstName>Bob</firstName>
out:        <lastName>Barnaby</lastName>
out:        <userid>bbarnaby</userid>
out:        <associatedGroups>
out:          <userGroup>
out:            <name>Standard CCM End Users</name>
out:          </userGroup>
out:          <userGroup>
out:            <name>Standard CTI Enabled</name>
out:          </userGroup>
out:        </associatedGroups>
out:        <presenceGroupName>Standard Presence group</presenceGroupName>
out:      </user>
out:    </ns0:addUser>
out:  </soap-env:Body>
out:</soap-env:Envelope>
in:<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/">
in:  <soapenv:Body>
in:    <soapenv:Fault>
in:      <faultcode>soapenv:Client</faultcode>
in:      <faultstring>Could not insert new row - duplicate value in a UNIQUE INDEX colu

Fault: Could not insert new row - duplicate value in a UNIQUE INDEX column (Unique Index:).

The SOAP response clearly shows that the same user can not be added twice. Zeep raised an exception after receiving the soapenv:Fault. 

Python allows us to intercept exceptions and gracefully act on them

In [28]:
try:
    r = service.addUser(user=new_user)
except zeep.exceptions.Fault as fault:
    # catch zeep exception. The Fault object has the AXL errors message and the detail element
    # from which we can extract the integer AXL error code
    axl_code = int(fault.detail.find('.//axlcode').text)
    print(f"Failed to insert user: {axl_code}/{fault.message}")
else:
    print(f'Successfully added user. UUID: {r["return"]}')


out:<soap-env:Envelope xmlns:soap-env="http://schemas.xmlsoap.org/soap/envelope/">
out:  <soap-env:Body>
out:    <ns0:addUser xmlns:ns0="http://www.cisco.com/AXL/API/11.5">
out:      <user>
out:        <firstName>Bob</firstName>
out:        <lastName>Barnaby</lastName>
out:        <userid>bbarnaby</userid>
out:        <associatedGroups>
out:          <userGroup>
out:            <name>Standard CCM End Users</name>
out:          </userGroup>
out:          <userGroup>
out:            <name>Standard CTI Enabled</name>
out:          </userGroup>
out:        </associatedGroups>
out:        <presenceGroupName>Standard Presence group</presenceGroupName>
out:      </user>
out:    </ns0:addUser>
out:  </soap-env:Body>
out:</soap-env:Envelope>
in:<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/">
in:  <soapenv:Body>
in:    <soapenv:Fault>
in:      <faultcode>soapenv:Client</faultcode>
in:      <faultstring>Could not insert new row - duplicate value in a UNIQUE INDEX colu